 计算机视觉（25春）作业1-4 (25分)
---

## 题目: DLT算法的使用
- **给定理想二维匹配点对集合，请使用DLT算法估计其所对应的单应性矩阵H。**
- 二维匹配点数据文件名为：2D_correspondences4DLT_H_25.txt
    - 存储格式为：每行4个数，分别为 (x1, x2, y1, y2)，表示两个平面间的匹配点坐标。
- 请在下面的代码框中编写DLT算法的实现，并输出估计得到的单应性矩阵H。

---
### 环境配置：以anaconda为例

```shell
    # anaconda的安装请根据自己的操作系统下载对应的安装包，官网链接：https://www.anaconda.com/products/distribution
    # 具体安装过程可以参考知乎，b站，CSDN等网站上的安装教程
    # 假设你已经成功安装了anaconda，打开terminal（linux/mac）or Anaconda Prompt（windows）
    cd <你的文件夹路径>
    conda create -n cv python=3.8 # 创建python虚拟环境，名字为cv，python版本为3.8
    conda activate cv # 激活虚拟环境
    conda install jupyter notebook # 安装jupyter notebook
    pip install --upgrade pip # 升级pip
    pip install opencv-python # 安装opencv
    pip install numpy # 安装numpy
    jupyter notebook # 运行jupyter notebook
    # 上一条指令将在默认的网页浏览器中开启一个新的工作空间，你可以在其中打开该.ipynb文件
```

---

## 数据读取与展示部分
- 请在下面的代码框中实现对数据的读取与展示。
- 评分细则：
    1.数据读取与展示的正确性。（5分）

In [13]:
# Code here
import numpy as np
data = np.loadtxt('2D_correspondences4DLT_H_25.txt')
num=data.shape[0]
print("Number of rows:")
print(num)
print("5 row of data:")
print(data[:5])

# hint: use np.loadtxt

Number of rows:
100
5 row of data:
[[98.27726696  0.88381594 68.56320055  3.52420276]
 [46.79868548  0.92020821 29.41655793  3.48053121]
 [77.47361618  0.78093792 73.74343828  3.62680435]
 [94.91709971  1.29491575 19.37235034  3.10858387]
 [83.8524972   0.99361846 42.69456762  3.41207569]]


---

## 数据处理部分
- 请在下面的代码框中实现数据的预处理，主要为齐次坐标的构造，并展示你的处理结果。
- 评分细则：
    1. 齐次坐标构造是否正确（5分）

In [31]:
# Code here
P1=[]
P2=[]

for axis in data:
    P1.append([axis[0], axis[2],1])
    P2.append([axis[1], axis[3],1])

P1_arr = np.array(P1, dtype=np.float64)   
P2_arr = np.array(P2, dtype=np.float64) 

print("P1_arr:")
print(P1_arr[:5])
print("P2_arr:")
print(P2_arr[:5])


P1_arr:
[[98.27726696 68.56320055  1.        ]
 [46.79868548 29.41655793  1.        ]
 [77.47361618 73.74343828  1.        ]
 [94.91709971 19.37235034  1.        ]
 [83.8524972  42.69456762  1.        ]]
P2_arr:
[[0.88381594 3.52420276 1.        ]
 [0.92020821 3.48053121 1.        ]
 [0.78093792 3.62680435 1.        ]
 [1.29491575 3.10858387 1.        ]
 [0.99361846 3.41207569 1.        ]]


--- 

## DLT算法实现部分
- 请在下面的代码框中实现DLT算法，并输出估计得到的单应性矩阵H。
- 注意：
    1. 你可以直接使用cv2.findHomography()函数，如果采用此方法，请在下面的Markdown框中附上手算的过程和结果，并对比两种方法的结果。
        - 评分细则：
            - opencv函数调用是否正确（5分）
    2. 你也可以自己实现DLT算法，此时则**不需要**附上手算的过程和结果。
        - 评分细则：
            - DLT算法实现是否正确（10分）
            - 结果是否正确（5分）

---
*如果直接调用cv2.findHomography()函数，请在此处附上手算的过程和结果：*
- 如有需要，请熟悉Latex和Markdown的相关语法，方便书写数学公式和文字说明。
- 评分细则：
    - 算法流程正确性（4分）
    - SVD分解结果正确性（4分）
    - 两种方法结果一致性（2分）
- 以下为答题区域，可以使用多个Markdown或者Code单元格

---

In [41]:
# Code here
def normalize_points(pts):
    """
    pts: (N,2) 原始点
    返回: pts_norm: (N,2), T: (3,3) 归一化矩阵
    """
    mean = pts.mean(axis=0)
    pts_centered = pts - mean
    avg_dist = np.mean(np.linalg.norm(pts_centered, axis=1))
    scale = np.sqrt(2) / avg_dist

    T = np.array([[scale,     0, -scale*mean[0]],
                  [    0, scale, -scale*mean[1]],
                  [    0,     0,              1]])
    ones = np.ones((pts.shape[0],1))
    pts_h = np.hstack([pts, ones])
    pts_norm_h = (T @ pts_h.T).T
    return pts_norm_h[:, :2], T


def compute_homography_dlt(pts1, pts2):
    """
    pts1,pts2: (N,2) 归一化后的对应点，N>=4
    返回: H (3×3) 归一化空间下的 Homography
    """
    N = pts1.shape[0]
    A = np.zeros((2*N, 9))
    for i, (x1, x2) in enumerate(zip(pts1, pts2)):

        A[2*i] = [x1[0], x1[1], 1, 0, 0, 0, -x2[0]*x1[0], -x2[0]*x1[1], -x2[0]]
        A[2*i+1] = [0, 0, 0, x1[0], x1[1], 1, -x2[1]*x1[0], -x2[1]*x1[1], -x2[1]]
    _, _, Vt = np.linalg.svd(A)
    h = Vt[-1]
    return h.reshape(3,3)

pts1, T1 = normalize_points(P1_arr[:, :2])
pts2, T2 = normalize_points(P2_arr[:, :2])

H1 = compute_homography_dlt(P1_arr, P2_arr)
H2 = compute_homography_dlt(pts1, pts2)
H2 = np.linalg.inv(T2) @ H2 @ T1

print("H1:")
print(H1)
print("H2:")
print(H2)


print("取第一个点验证：")
p2=H2@P1_arr[0]
print("P1_arr[0]:")
print(P1_arr[0])
print("p2:")
print(p2/ p2[2])
print("P2_arr[0]:")
print(P2_arr[0])



H1:
[[-0.18041973 -0.06193671 -0.10569404]
 [-0.28611376 -0.87113027 -0.21138808]
 [-0.10569404 -0.21138808 -0.10569404]]
H2:
[[-0.00631692 -0.00216855 -0.0037006 ]
 [-0.01001752 -0.03050034 -0.0074012 ]
 [-0.0037006  -0.0074012  -0.0037006 ]]
取第一个点验证：
P1_arr[0]:
[98.27726696 68.56320055  1.        ]
p2:
[0.88381594 3.52420276 1.        ]
P2_arr[0]:
[0.88381594 3.52420276 1.        ]
